In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import sys
import os
from sklearn.model_selection import train_test_split

from tkinter import *
import time
'''Tensorflow package'''
# from tensorflow.contrib.slim.nets import resnet_v2
## shuffle比較容易訓練
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

#進度條
from ipywidgets import IntProgress
from IPython.display import display

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Source Path
# 設定圖片路徑, 分類路徑

In [2]:
class_config = "data/mapping.txt"
# img_train_path = "data/train/"
#裁切圖片完成的路徑
img_train_path = "data/train_cut_v6_1/"
img_test_path = "data/test_cut_v6_1/"
df_classes = pd.read_csv(os.path.join(class_config))
df_classes

,classes,dirs
0,0,Pass
1,1,Missing
2,2,MissAlignment
3,3,Tombstone_SideStanding
4,4,PartsReversed
5,5,ExcessComponent
6,6,Other_Damage
7,7,VoidSolder_Short
8,8,TextMissing
9,9,tag


## 建立圖片位置檔案

In [4]:
#紀錄圖片名稱
x_class=[]
#紀錄圖片路徑
x_path=[]
#r紀錄目錄
x_dir=[]

In [5]:
progress = IntProgress()
progress.max = 40000
progress.description = '(Init)'
display(progress)
time.sleep(0.25)

tStart = time.time()#計時開始

for index, row in df_classes.iterrows():
    scr_directory = img_train_path+row['dirs']+'/'
    print(scr_directory)
    if os.path.isdir(scr_directory):
        for imgname in os.listdir(scr_directory):
            dir_img = scr_directory + imgname
            train_img =cv2.imread(dir_img)
            #防呆,避免讀取的不是圖檔
            if train_img is not None:
                imgname_split=imgname.split('.') 
                if imgname_split != '':
                    x_class.append(row['classes'])
                    path = scr_directory+imgname
                    x_path.append(path)
                    x_dir.append(row['dirs'])

                    progress.value += 1
                    time.sleep(0.05)
                    progress.description = 'Training Image'


tEnd = time.time()#計時結束
print("It cost %f sec" % (tEnd - tStart))#會自動做近位


IntProgress(value=0, description='(Init)', max=40000)

data/train_cut_v6_1/Pass/
data/train_cut_v6_1/Missing/
data/train_cut_v6_1/MissAlignment/
data/train_cut_v6_1/Tombstone_SideStanding/
data/train_cut_v6_1/PartsReversed/
data/train_cut_v6_1/ExcessComponent/
data/train_cut_v6_1/Other_Damage/
data/train_cut_v6_1/VoidSolder_Short/
data/train_cut_v6_1/TextMissing/
data/train_cut_v6_1/tag/
It cost 1117.386139 sec


In [6]:
#將圖檔和路徑合併
# x_table = pd.DataFrame(x_name)
x_class = pd.DataFrame(x_class)
x_class.rename(columns={ x_class.columns[0]: "class"}, inplace=True)

x_path = pd.DataFrame(x_path)
x_path.rename(columns={ x_path.columns[0]: "img_path"}, inplace=True)

x_dir = pd.DataFrame(x_dir)
x_dir.rename(columns={ x_dir.columns[0]: "dir_name"}, inplace=True)

# x_table = pd.concat(x_name,x_path)
x_table = pd.concat([x_path,x_class,x_dir],axis=1)



In [23]:
x_table.head()

,img_path,class,dir_name
0,data/train_cut_v5_1/Pass/0915104416_AOI_E02-0R...,0,Pass
1,data/train_cut_v5_1/Pass/0910192346_AOI_E01-0C...,0,Pass
2,data/train_cut_v5_1/Pass/0915171815_AOI_E02-0R...,0,Pass
3,data/train_cut_v5_1/Pass/0907075829_AOI_E02-0D...,0,Pass
4,data/train_cut_v5_1/Pass/0918151025_AOI_E02-0R...,0,Pass


In [ ]:
# x_table.to_csv('x_table.csv', index = False)

In [7]:
x_table = x_table.drop(axis=1, columns='dir_name')
x_dummies = pd.get_dummies(x_table["class"])

In [8]:
# #組合圖片路徑 跟 get_dummies資料組合
x_table = pd.concat([x_table,x_dummies],axis=1)
x_table = x_table.drop(["class"], axis = 1)

In [9]:
x_table.head(5)

,img_path,0,1,2,3,4,5,6,7,8,9
0,data/train_cut_v6_1/Pass/0915104416_AOI_E02-0R...,1,0,0,0,0,0,0,0,0,0
1,data/train_cut_v6_1/Pass/0910192346_AOI_E01-0C...,1,0,0,0,0,0,0,0,0,0
2,data/train_cut_v6_1/Pass/0915171815_AOI_E02-0R...,1,0,0,0,0,0,0,0,0,0
3,data/train_cut_v6_1/Pass/0907075829_AOI_E02-0D...,1,0,0,0,0,0,0,0,0,0
4,data/train_cut_v6_1/Pass/0918151025_AOI_E02-0R...,1,0,0,0,0,0,0,0,0,0


In [10]:
#儲存整理好的資料
x_table.to_csv("train_data_cut_v6_1.csv", index = False)


## 如果重新Trainning 從這裡開始執行
## 讀取 已經紀錄的圖片路徑，是已經get dummie

In [11]:
x_table = pd.read_csv('train_data_cut_v6_1.csv')

In [31]:
x_table.head(5)

,img_path,0,1,2,3,4,5,6,7
0,data/train_cut_v2/Pass/0915104416_AOI_E02-0R12...,1,0,0,0,0,0,0,0
1,data/train_cut_v2/Pass/0914161127_AOI_E01-0C10...,1,0,0,0,0,0,0,0
2,data/train_cut_v2/Pass/0904093307_AOI_E01-0C10...,1,0,0,0,0,0,0,0
3,data/train_cut_v2/Pass/0910192346_AOI_E01-0C13...,1,0,0,0,0,0,0,0
4,data/train_cut_v2/Pass/0915171815_AOI_E02-0R05...,1,0,0,0,0,0,0,0


# Shuffle 


In [12]:
#打亂
x_table = shuffle(x_table)


In [13]:
x_table.to_csv('X-Table_shuffle_cut_v6_1.csv', index = False)

# split train data
# 取打亂的圖片來源檔

In [3]:
x_table = pd.read_csv('X-Table_shuffle_cut_v5_1.csv')

In [14]:
x_table.head()

,img_path,0,1,2,3,4,5,6,7,8,9
12658,data/train_cut_v6_1/Pass/0913075935_AOI_E01-0R...,1,0,0,0,0,0,0,0,0,0
3331,data/train_cut_v6_1/Pass/0916014959_AOI_E02-0R...,1,0,0,0,0,0,0,0,0,0
15022,data/train_cut_v6_1/Missing/0821132924_AOI_E02...,0,1,0,0,0,0,0,0,0,0
5252,data/train_cut_v6_1/Pass/0917181520_AOI_E02-0C...,1,0,0,0,0,0,0,0,0,0
7441,data/train_cut_v6_1/Pass/0914052402_AOI_E01-0C...,1,0,0,0,0,0,0,0,0,0


In [15]:
X = x_table['img_path']
y = x_table.drop("img_path", axis = 1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25)

建立generator

In [3]:
from keras.preprocessing.image import ImageDataGenerator

## 設定config
img_size_H = 250
img_size_W = 200
img_size = 200
num_class = 10
batch_size = 24
channel_num = 3
epoch = 50

Using TensorFlow backend.


In [22]:
#Generator
x = []

img_gen = ImageDataGenerator(
    rotation_range=0.0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.0,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='constant',
    cval=0)


def generator_input(data_path, size = img_size):    
    x = []
    for path in data_path:
        img = cv2.imread(path)
        img = cv2.resize(img, (img_size_H, img_size_W))
#         img = img[:,:,::-1] # convert BGR to RGB  ##test turn it off
        img = img.astype('float32') - np.array([123.68, 116.78, 103.94]) 
        x.append(img)
    
    x = np.array(x)
    return x





In [23]:
if not os.path.exists('saved_models/'):
    os.makedirs('saved_models')

In [24]:
import keras
from keras.models import Sequential, load_model, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.resnet50 import ResNet50


In [25]:
##model##
#如果錯誤訊息是 OOM 代表圖片太大

model_name = 'ResNet50-Fine-Tuning_cut_v6_1'


base_model = ResNet50(weights='imagenet', include_top = False, 
                      input_shape=(img_size_W, img_size_H, 3))


x = base_model.output
# x = Flatten()(x) #不用Flatten,改用GlobaAveragePoooling2D
x = GlobalAveragePooling2D(data_format='channels_last')(x)

x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(256, activation='relu')(x)

predictions = Dense(num_class, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# model.summary()
optimizer = keras.optimizers.Adam(lr=10e-6) 

model_path = 'saved_models/{}.h5'.format(model_name)

checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])


aug_ratio = 1
steps_per_epoch = int(aug_ratio * X_train.shape[0] / batch_size)
validation_steps = int(aug_ratio * X_valid.shape[0] / batch_size)

model_history = model.fit_generator(img_gen.flow(generator_input(X_train), y_train, batch_size = batch_size),
                                    epochs = epoch,
                                    validation_data = (generator_input(X_valid), y_valid),
                                    callbacks = [checkpoint, earlystop],
                                    steps_per_epoch = steps_per_epoch,
                                    validation_steps = validation_steps)


/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Epoch 1/50
610/610 [==============================] - 148s 242ms/step - loss: 0.8284 - acc: 0.7836 - val_loss: 0.3474 - val_acc: 0.9199

Epoch 00001: val_loss improved from inf to 0.34739, saving model to saved_models/ResNet50-Fine-Tuning_cut_v6_1.h5
Epoch 2/50
610/610 [==============================] - 136s 223ms/step - loss: 0.3485 - acc: 0.9155 - val_loss: 0.3002 - val_acc: 0.9199

Epoch 00002: val_loss improved from 0.34739 to 0.30021, saving model to saved_models/ResNet50-Fine-Tuning_cut_v6_1.h5
Epoch 3/50
610/610 [==============================] - 136s 223ms/step - loss: 0.2981 - acc: 0.9215 - val_loss: 0.2486 - val_acc: 0.9295

Epoch 00003: val_loss improved from 0.30021 to 0.24859, saving model to saved_models/ResNet50-Fine-Tuning_cut_v6_1.h5
Epoch 4/50
610/610 [==============================] - 136s 223ms/step - loss: 0.2560 - acc: 0.9283 - val_loss: 0.2242 - val_acc: 0.9336

Epoch 00004: val_loss improved from 0.24859 to 0.22424, saving model to saved_models/ResNet50-Fine-Tun

# @ testset 預測答案

In [6]:
## 設定config
img_size_H = 250
img_size_W = 200
img_size = 200
num_class = 10
batch_size = 24
channel_num = 3
epoch = 50

In [4]:
y_table = pd.read_csv("data/test_cut_v6_1.csv")

In [ ]:
# orig_test.head(10)

In [10]:
y_table.head(5)

,img_path,class
0,0913043348_AOI_E01-0R027_4_WhiteLight.jpg,2
1,0917120507_AOI_E02-0R038_3_LowAngleLight.jpg,0
2,0809191038_AOI_E02-0R040_2_SolderLight.jpg,1
3,0917153925_AOI_E02-0R130_1_SolderLight.jpg,0
4,0917140111_AOI_E02-0C148_3_SolderLight.jpg,0


In [ ]:
# img_test_path
#紀錄圖片名稱
x_class=[]
#紀錄圖片路徑
x_path=[]
#r紀錄目錄
x_dir=[]
i = 1


for imgname in os.listdir(img_test_path):
    i+=1
    dir_img = img_test_path + imgname
    train_img =cv2.imread(dir_img)
#     #防呆,避免讀取的不是圖檔
    if train_img is not None:
        df_name = orig_test[orig_test['img_path'] == imgname]     
        class_name = df_name['class']
        if(class_name.count != 0):
            class_name = int(class_name.iloc[0])
            x_class.append(class_name)
            path = imgname
            x_path.append(path)

print('count: ',i)



In [9]:

def generator_test(data_path, size = img_size):    
    x = []
    for path in data_path:
        img = cv2.imread(img_test_path + path)
        img = cv2.resize(img, (img_size_H, img_size_W))
                
#         img = img[:,:,::-1] # convert BGR to RGB  ##test turn it off
        img = img.astype('float32') - np.array([123.68, 116.78, 103.94]) 
        x.append(img)
    
    x = np.array(x)
    return x

In [9]:
#將圖檔和路徑合併
# x_table = pd.DataFrame(x_name)
# x_class = pd.DataFrame(x_class)
# x_class.rename(columns={ x_class.columns[0]: "class"}, inplace=True)

# x_path = pd.DataFrame(x_path)
# x_path.rename(columns={ x_path.columns[0]: "img_path"}, inplace=True)

# x_dir = pd.DataFrame(x_dir)
# x_dir.rename(columns={ x_dir.columns[0]: "dir_name"}, inplace=True)

# y_table = pd.concat([x_path,x_class],axis=1)
# y_table = pd.concat([x_path,x_class],axis=1)

In [10]:
# y_table.to_csv('data/test_date_cut_v5_1.csv', index = False)

In [15]:
y_table.head(5)

,img_path,class
0,0915121849_AOI_E02-0R077_4_UniformLight.jpg,5
1,0919150453_AOI_E02-0R080_4_UniformLight.jpg,0
2,0919144541_AOI_E02-0R038_4_WhiteLight.jpg,0
3,0919105040_AOI_E02-0C063_4_WhiteLight.jpg,5
4,0919151324_AOI_E02-0R016_3_WhiteLight.jpg,0


In [5]:
import cv2
# y_temp = pd.read_csv("data/1004-test_data.csv")
# y_temp = pd.read_csv("data/test_date_cut_v5_1.csv")
# y_temp = y_temp['filename']
y_temp = y_table['img_path']

In [6]:
y_temp.head(5)

0       0913043348_AOI_E01-0R027_4_WhiteLight.jpg
1    0917120507_AOI_E02-0R038_3_LowAngleLight.jpg
2      0809191038_AOI_E02-0R040_2_SolderLight.jpg
3      0917153925_AOI_E02-0R130_1_SolderLight.jpg
4      0917140111_AOI_E02-0C148_3_SolderLight.jpg
Name: img_path, dtype: object

In [10]:
y_gen_table = generator_test(y_temp, size = img_size)

In [11]:
from keras.models import Sequential, load_model, Model

model = load_model('saved_models/ResNet50-Fine-Tuning_cut_v6_1.h5')
y_pred = model.predict(y_gen_table)
y_pred = y_pred.argmax(axis = 1)
y_pred = pd.DataFrame(y_pred)
#合併id檔
result = pd.concat([y_temp,y_pred], axis = 1)


In [12]:
result.head()

,img_path,0
0,0913043348_AOI_E01-0R027_4_WhiteLight.jpg,5
1,0917120507_AOI_E02-0R038_3_LowAngleLight.jpg,0
2,0809191038_AOI_E02-0R040_2_SolderLight.jpg,1
3,0917153925_AOI_E02-0R130_1_SolderLight.jpg,0
4,0917140111_AOI_E02-0C148_3_SolderLight.jpg,0


In [13]:
#移除class欄位
# result = result.drop(["class"], axis = 1)
#rename columns name
result.rename(columns={result.columns[1]: "class"}, inplace=True)

In [14]:
result.head(5)

,img_path,class
0,0913043348_AOI_E01-0R027_4_WhiteLight.jpg,5
1,0917120507_AOI_E02-0R038_3_LowAngleLight.jpg,0
2,0809191038_AOI_E02-0R040_2_SolderLight.jpg,1
3,0917153925_AOI_E02-0R130_1_SolderLight.jpg,0
4,0917140111_AOI_E02-0C148_3_SolderLight.jpg,0


In [15]:
#把答案跟預測比對
# y_temp = pd.read_csv("data/test_date_cut_v5_1.csv")
y_result = pd.concat([result,y_table],axis=1)
y_result.to_csv('img_result_cut_v6_1.csv', index = False)

In [34]:
y_pred.head(5)

,0
0,5
1,0
2,0
3,5
4,0


In [21]:
y_table_class = y_table['class']

In [22]:
from sklearn.metrics import confusion_matrix
C = confusion_matrix(y_table_class, y_pred)

In [39]:
df_classes

,classes,dirs
0,0,Pass
1,1,Missing
2,2,MissAlignment
3,3,Tombstone_SideStanding
4,4,PartsReversed
5,5,ExcessComponent
6,6,TextMissing
7,7,VoidSolder_Short


In [23]:
print(C)

[[566   0   0   0   1   1   1   5   1   0]
 [  0 106   0   0   0   0   0   0   0   0]
 [  0   0   1   0   0   3   0   0   0   0]
 [  0   0   0   4   0   0   0   0   0   0]
 [  0   0   0   0   7   0   0   0   0   0]
 [  8   0   0   0   0   0   0   1   0   2]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   5   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]


In [83]:
# result.to_csv('img_result.csv', index = False)

In [34]:
#建立放置分類圖片目錄
test_dir_name = 'data/test_classes'
if not os.path.exists(test_dir_name):
    os.makedirs(test_dir_name)

#取test pos
df_map = pd.read_csv('data/mapPos.csv');

In [35]:
# 把testset 預測出來的答案，把檔案轉到分類目錄。
import shutil

dir_name = ""
pic_name = ""

for index, row in result.iterrows():
    df_name = df_classes[df_classes['classes'] == row['class']]
#     df_classes = row
#     print(row['filename'])
    class_name = df_name['dirs']
    
    dir_name = 'data/test_classes/'+ class_name.iloc[0] + '/'
    #建立分類目錄
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

    #將圖片copy 到對應的目錄
    shutil.copy('data/testset/'+row['filename'],dir_name)
